In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import differential_entropy,kurtosis
import pandas as pd
from scipy.optimize import minimize
import statsmodels.api as sm

In [2]:
def Majew(gamma, alpha, kappa1, beta, Lambda, sigmav, length = 1000, nb = 1 ):
    #nb : nb de simulations
    # length : taille de la serie temporelle
    Epsilon=np.random.normal(0, sigmav**2, (length,nb))
    v=np.zeros((length,nb))
    m=np.zeros((length,nb))
    
    # We initialize the different functions we will need
    
    p=np.zeros((length,nb))
    r=np.zeros((length,nb))
    # We initialize the price

    for t in range(1,length-1):
        
        v[t] = (1-Lambda)*v[t-1] + Lambda*p[t]
        
        m[t] = (1-alpha)*m[t-1] + alpha*(p[t]-p[t-1])
        
        p[t+1] = p[t] + kappa1*(v[t]-p[t]) + beta*np.tanh(gamma*m[t]) + Epsilon[t+1]
        
        r[t+1] = p[t+1] - p[t]
    
    return p,r

In [18]:
def compute_statistics(r, cov=False):
    means = np.nanmean(r, axis=0)
    std_dev = np.nanstd(r, axis=0)
    kurtosiss = kurtosis(r, axis=0)
    
    abs_r = np.abs(r)
    q1 = np.nanpercentile(abs_r, 10, axis=0)
    q2 = np.nanpercentile(abs_r, 95, axis=0)
    rat_quantile = q1/q2
    if r.shape[1] == 1 :
        aut_r = sm.tsa.acf(r, nlags= 1)[1]
        aut_rc = sm.tsa.acf(r**2, nlags= 1)[1]
    else :
        deltar = (r-np.nanmean(r,axis=0))/np.nanstd(r,axis=0)
        aut_r = np.array([sm.tsa.acf(deltar[:, i], nlags=7)[1] for i in range(r.shape[1])])
        
        aut_rc = np.array([sm.tsa.acf((deltar[:, i])**2, nlags=7)[1] for i in range(r.shape[1])])
        # if (np.sum(np.isnan(aut_r)) > 0 ):
        #     print("danger1")
        # if  (np.sum(np.isnan(aut_rc)) > 0 ) : 
        #     print("danger2")
    
    if cov:
        mf = np.vstack([means, std_dev, kurtosiss, rat_quantile, aut_r, aut_rc])
        # if np.sum(np.isnan(mf)) > 0 :
        #     print("danger")
        W = np.cov(mf)
        # if np.sum(np.isnan(W)) > 0 :
        #     print("danger")
        
        return np.array([np.nanmean(means), np.nanmean(std_dev), np.nanmean(kurtosiss), np.nanmean(rat_quantile), np.nanmean(aut_r), np.nanmean(aut_rc)]), W
    
    return np.array([np.nanmean(means), np.nanmean(std_dev), np.nanmean(kurtosiss), np.nanmean(rat_quantile), np.nanmean(aut_r), np.nanmean(aut_rc)])



In [19]:
_, r_thild = Majew(36.7,1/7,0.015,0.015,0.05,0.018)
c_thilde = compute_statistics(r_thild)
c_thilde

array([-1.78125150e-05,  3.22080146e-04,  7.00192682e-02,  6.51153396e-02,
        5.39389365e-02, -9.47329765e-03])

In [6]:
def cost(ksi):
    _, r = Majew(*ksi, nb = 100)
    if(np.sum(np.isnan(r))) : 
        print("danger")
    cabm, V = compute_statistics(r, cov=True)
    
    dc = c_thilde - cabm
    W = np.linalg.inv(V)
    D = dc.T @ W @ dc
    print(D)
    return D

In [7]:
options = {
    'maxiter': 1000, 
    'disp': True,
    'adaptive': False
}
init = (30.7,1/5,0.095,0.005,0.2,0.09)
bounds = [(36.7*0.3,36.7*1.7),((1/7)*0.3,(1/7)*1.7),(0.015*0.3,0.015*1.7),(0.015*0.3,0.015*1.7),(0.1*0.3,0.1*1.7),(0.018*0.3,0.018*1.7)]
res = minimize(cost, init, method='Powell', options=options, bounds=bounds)


/tmp/ipykernel_1303/922999621.py:8: OptimizeWarning: Unknown solver options: adaptive
  res = minimize(cost, init, method='Powell', options=options, bounds=bounds)
/tmp/ipykernel_1303/922999621.py:8: OptimizeWarning: Initial guess is not within the specified bounds
  res = minimize(cost, init, method='Powell', options=options, bounds=bounds)


2653.1596987722332
1864.864494535278
1754.3535326909202
1954.0244050280533
2222.7628086550512
1868.8138417874704
2610.5938060792614
2038.969293212733
1940.20131870441
2530.6500921742545
2205.1408796971946
1962.6948258803686
1955.7560896144
1558.5224463198824
1881.8635793364413
1895.791847058029
2006.7509986489404
1714.7652363023283
1944.7369079464672
1769.2511354652104
2386.7884023832626
2100.0459224652186
1778.2655442420069
2309.5090635689376
1834.0930911134321
2250.4758042387343
2324.4331684983486
2043.7365476255645
1988.9825455461846
1995.9958300774392
1885.8141824564038
1580.9105660526814
1864.9584558954384
1478.2582859811823
2343.67646259622
1627.8464243953626
1891.596230761307
2057.0821379101544
1448.1980736963276
1957.1595406314361
2162.4494746676937
1654.3158170190595
1724.652999284947
1845.1708392836858
1749.2639076820733
1480.6679399159011
2113.6088262990766
1756.0794963684493
2286.4865243774357
2059.961963794267
1777.0334674130609
1776.0795395299926
2206.992511658069
2036.84

In [8]:
params = np.array([06.7, 1/10, 0.5, 0.8, 0.3, 0.5])
res.x

array([3.54181215e+01, 1.27053534e-01, 1.20742753e-02, 1.54075165e-02,
       1.15048006e-01, 1.77791477e-02])

In [12]:
params = np.array([36.7,1/7,0.015,0.015,0.05,0.018])

percentage_range = 0.5  # 10% range
bounds = [(param * (1 - percentage_range), param * (1 + percentage_range)) for param in params]
_, r_thild = Majew(*params)
c_thilde = compute_statistics(r_thild)


def cost(ksi):
    _, r = Majew(*ksi, nb = 100)
    if(np.sum(np.isnan(r))) : 
        print("danger")
    cabm, V = compute_statistics(r, cov=True)
    dc = c_thilde - cabm
    #W = np.linalg.inv(V)
    D = dc.T @ W @ dc
    return D

options = {
    'maxiter': 1000, 
    'disp': True
}
epsilon = 0.1
ksi = [np.random.uniform(low,high) for low,high in bounds]
_, r = Majew(*init, nb = 100)
cabm, V = compute_statistics(r, cov=True)
W = np.linalg.inv(V)
i = 1
while True :
    res = minimize(cost, ksi, method='Powell',bounds=bounds, options=options)
    error = np.linalg.norm(res.x -ksi)
    print(f"resultat etape {i} : ", error)
    if (error < 0.1) or (i>=100):
        break
    ksi=res.x
    _, r = Majew(*ksi, nb = 100)
    cabm, V = compute_statistics(r, cov=True)
    W = np.linalg.inv(V)
    i = i+1


Optimization terminated successfully.
         Current function value: 2.208648
         Iterations: 2
         Function evaluations: 161
resultat etape 1 :  10.229568636746096
Optimization terminated successfully.
         Current function value: 1.835471
         Iterations: 3
         Function evaluations: 252
resultat etape 2 :  0.12080145869593065
Optimization terminated successfully.
         Current function value: 2.124958
         Iterations: 2
         Function evaluations: 166
resultat etape 3 :  0.8026203541936636
Optimization terminated successfully.
         Current function value: 3.708338
         Iterations: 1
         Function evaluations: 91
resultat etape 4 :  0.547692220219992
Optimization terminated successfully.
         Current function value: 3.288589
         Iterations: 2
         Function evaluations: 160
resultat etape 5 :  5.009156796066505


KeyboardInterrupt: 

In [11]:
params = np.array([36.7,1/7,0.015,0.015,0.05,0.018])

percentage_range = 1  # 10% range
bounds = [(min(-2*param, 2*param), max(-2*param, 2*param)) for param in params]
_, r_thild = Majew(*params)
c_thilde = compute_statistics(r_thild)


def cost(ksi):
    _, r = Majew(*ksi, nb = 100)
    if(np.sum(np.isnan(r))) : 
        print("danger")
    cabm, V = compute_statistics(r, cov=True)
    dc = c_thilde - cabm
    #W = np.linalg.inv(V)
    D = dc.T @ W @ dc
    return D

options = {
    'maxiter': 1000, 
    'disp': True
}
epsilon = 0.1
ksi = [np.random.uniform(low,high) for low,high in bounds]
_, r = Majew(*init, nb = 100)
cabm, V = compute_statistics(r, cov=True)
W = np.linalg.inv(V)
i = 1
while True :
    res = minimize(cost, ksi, method='Powell',bounds=bounds, options=options)
    error = np.linalg.norm(res.x -ksi)
    print(f"resultat etape {i} : ", error)
    if (error < 0.1) or (i>=100):
        break
    ksi=res.x
    _, r = Majew(*ksi, nb = 100)
    cabm, V = compute_statistics(r, cov=True)
    W = np.linalg.inv(V)
    i = i+1


Optimization terminated successfully.
         Current function value: 13.199373
         Iterations: 4
         Function evaluations: 376
resultat etape 1 :  23.41788632771441
Optimization terminated successfully.
         Current function value: 11.960394
         Iterations: 4
         Function evaluations: 378
resultat etape 2 :  1.2421903191531454
Optimization terminated successfully.
         Current function value: 19.246648
         Iterations: 1
         Function evaluations: 99
resultat etape 3 :  2.3438294105345117
Optimization terminated successfully.
         Current function value: 15.313649
         Iterations: 3
         Function evaluations: 300
resultat etape 4 :  14.185675589644227
Optimization terminated successfully.
         Current function value: 19.175136
         Iterations: 2
         Function evaluations: 198
resultat etape 5 :  12.524826317573602
Optimization terminated successfully.
         Current function value: 14.684062
         Iterations: 4
        

In [13]:
np.linalg.norm(res.x - params)
res.x

array([3.62694620e+01, 1.62839403e-01, 1.35796648e-02, 1.02050983e-02,
       5.59016994e-02, 1.77965172e-02])

In [15]:
params = np.array([36.7,1/7,0.015,0.015,0.05,0.018])

percentage_range = 1  # 10% range
bounds = [(min(-2*param, 2*param), max(-2*param, 2*param)) for param in params]
_, r_thild = Majew(*params)
c_thilde = compute_statistics(r_thild)


def cost(ksi):
    _, r = Majew(*ksi, nb = 100)
    if(np.sum(np.isnan(r))) : 
        print("danger")
    cabm, V = compute_statistics(r, cov=True)
    dc = c_thilde - cabm
    #W = np.linalg.inv(V)
    D = dc.T @ W @ dc
    return D

options = {
    'maxiter': 1000, 
    'disp': True
}
epsilon = 0.1
ksi = [np.random.uniform(low,high) for low,high in bounds]
_, r = Majew(*init, nb = 100)
cabm, V = compute_statistics(r, cov=True)
W = np.linalg.inv(V)
i = 1
while True :
    res = minimize(cost, ksi, method='Powell',bounds=bounds, options=options)
    error = np.linalg.norm(res.x -ksi)
    print(f"resultat etape {i} : ", error)
    if (error < 0.1) or (i>=100):
        break
    ksi=res.x
    _, r = Majew(*ksi, nb = 100)
    cabm, V = compute_statistics(r, cov=True)
    W = np.linalg.inv(V)
    i = i+1



Optimization terminated successfully.
         Current function value: 3.140484
         Iterations: 5
         Function evaluations: 552
resultat etape 1 :  6.1817588335708615
Optimization terminated successfully.
         Current function value: 3.291388
         Iterations: 4
         Function evaluations: 378
resultat etape 2 :  4.538213908494617
Optimization terminated successfully.
         Current function value: 3.259854
         Iterations: 2
         Function evaluations: 201
resultat etape 3 :  12.699742621814774
Optimization terminated successfully.
         Current function value: 3.093207
         Iterations: 2
         Function evaluations: 189
resultat etape 4 :  1.2991687469679873
Optimization terminated successfully.
         Current function value: 3.030324
         Iterations: 4
         Function evaluations: 388
resultat etape 5 :  4.496171757896487
Optimization terminated successfully.
         Current function value: 2.233760
         Iterations: 2
         Funct

In [16]:
print(f' final error : {np.linalg.norm(res.x - params)}')
print(f'final parameter : {res.x}  \nreal parameter : {params}')

 final error : 71.74120629019588
final parameter : [-3.50411632e+01  2.02558893e-01  7.44878946e-03 -1.75917230e-02
  3.62798440e-02 -1.81460252e-02]  
real parameter : [3.67000000e+01 1.42857143e-01 1.50000000e-02 1.50000000e-02
 5.00000000e-02 1.80000000e-02]


In [ ]:
def cost(ksi, W):
    _, r = Majew(*ksi, nb = 100)
    if(np.sum(np.isnan(r))) : 
        print("danger")
    cabm, V = compute_statistics(r, cov=True)
    dc = c_thilde - cabm
    #W = np.linalg.inv(V)
    D = dc.T @ W @ dc
    return D

def find_optimalksi(params= [36.7,1/7,0.015,0.015,0.05,0.018], cost = cost, Majew = Majew, epsilon = 0.1, maxiter = 100, print = True, dis = True) :

    percentage_range = 1  # 10% range
    bounds = [(min(-2*param, 2*param), max(-2*param, 2*param)) for param in params]
    _, r_thild = Majew(*params)
    c_thilde = compute_statistics(r_thild)
    options = {
        'maxiter': 1000, 
        'disp': dis
    }
    ksi = [np.random.uniform(low,high) for low,high in bounds]
    _, r = Majew(*ksi, nb = 100)
    cabm, V = compute_statistics(r, cov=True)
    W = np.linalg.inv(V)
    i = 1
    while True :
        res = minimize(cost, ksi, args=W, method='Powell',bounds=bounds, options=options)
        error = np.linalg.norm(res.x -ksi)
        if print : 
            print(f"resultat etape {i} : ", error)
        if (error < 0.1) or (i>=maxiter):
            break
        ksi=res.x
        _, r = Majew(*ksi, nb = 100)
        cabm, V = compute_statistics(r, cov=True)
        W = np.linalg.inv(V)
        i = i+1

    return r_thild, c_thilde, ksi

In [35]:
r_thild, c_thilde, ksi = find_optimalksi()

Optimization terminated successfully.
         Current function value: 325123249499390410752.000000
         Iterations: 2
         Function evaluations: 215
resultat etape 1 :  5.781919386196894
Optimization terminated successfully.
         Current function value: 2.345923
         Iterations: 3
         Function evaluations: 293
resultat etape 2 :  36.293202231292234
Optimization terminated successfully.
         Current function value: 6.808341
         Iterations: 2
         Function evaluations: 165
resultat etape 3 :  80.79233709341719
Optimization terminated successfully.
         Current function value: 2.759324
         Iterations: 1
         Function evaluations: 89
resultat etape 4 :  6.0514772398106444
Optimization terminated successfully.
         Current function value: 2.197414
         Iterations: 3
         Function evaluations: 286
resultat etape 5 :  2.436491899905525
Optimization terminated successfully.
         Current function value: 1.128055
         Iterations

In [86]:
def Majew2(gamma, alpha, kappa1, kappa3, beta, Lambda, sigmav):
    
    Epsilon=np.random.normal(0, sigmav**2, (length,nb))
    v=np.zeros((length,nb))
    m=np.zeros((length,nb))
    
    # We initialize the different functions we will need
    
    p=np.zeros((length,nb))
    r=np.zeros((length,nb))
    
    # We initialize the price
    
    for t in range(1,length-1):
        
        v[t] = (1-Lambda)*v[t-1] + Lambda*p[t]
        
        m[t] = (1-alpha)*m[t-1] + alpha*(p[t]-p[t-1])
        
        p[t+1] = p[t] + kappa1*(v[t]-p[t]) + kappa3*(v[t]-p[t]) + beta*np.tanh(gamma*m[t]) + Epsilon[t+1]
        
        r[t+1] = p[t+1] - p[t]
    
    return p,r